In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from battleship.board import Board
from battleship.agents import CodeQuestion, Question
from battleship.game import BattleshipGame
from battleship.captains import create_captain
from battleship.spotters import create_spotter
from battleship.fast_sampler import FastSampler

In [ ]:
import logging

logging.basicConfig(level=logging.DEBUG)


In [ ]:
BOARD_ID = "B03"
SEED = 123

true_board = Board.from_trial_id(BOARD_ID)
true_board

In [ ]:
def heatmap(board: Board, n_samples: int = 10000, constraints: list = []):
    sampler = FastSampler(
        board, ship_lengths=Board.SHIP_LENGTHS, ship_labels=Board.SHIP_LABELS, seed=SEED
    )
    # Compute posterior (return raw counts)
    posterior = sampler.compute_posterior(n_samples=n_samples, normalize=False, constraints=constraints)
    return Board._to_figure(posterior, mode="heatmap")

In [ ]:
spotter = create_spotter(
    spotter_type="CodeSpotterModel",
    board_id=BOARD_ID,
    board_experiment="collaborative",
    llm="gpt-5",
    use_cot=True,
    json_path=None,
)

captain = create_captain(
    captain_type="MAPCaptain",
    seed=SEED,
    llm="gpt-5",
    board_id=BOARD_ID,
    map_samples=1000,
    prob_q_prob=None,
    eig_samples=1000,
    eig_k=10,
    json_path=None,
)

game = BattleshipGame(
    board_target=true_board,
    captain=captain,
    spotter=spotter,
)

In [ ]:
game

In [ ]:
heatmap(game.state)

In [ ]:
while not game.is_done():
    game.next_stage()
    print(f"Stage {game.stage_index}")
    display(game)
    display(heatmap(game.state))
    break

In [ ]:
code_question = spotter.translate(
    question=Question("Is the red ship vertical?"),
    board=game.state,
    history=game.history,
)

In [ ]:
print(code_question.fn_str)

In [ ]:
answer = code_question(true_board=true_board.board, partial_board=game.state.board)
print(answer)

In [ ]:
heatmap(game.state, constraints=[(code_question, False)])